In [1]:
import requests
import os
from typing import Dict, Any, Optional

In [2]:
def predict_disease(api_url: str, image_path: str, fruit_type: str) -> Optional[Dict[str, Any]]:
    """
    Sends an image to the Plant Disease API and returns the prediction.

    Args:
        api_url: The URL of the prediction API endpoint.
        image_path: The local path to the image file.
        fruit_type: The type of fruit/crop depicted in the image.

    Returns:
        A dictionary containing the prediction results if successful, otherwise None.
    """
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at path: {image_path}")
        return None

    payload: Dict[str, str] = {'fruit': fruit_type}
    file_name: str = os.path.basename(image_path)

    try:
        with open(image_path, 'rb') as image_file:
            files_to_upload = {
                'file': (file_name, image_file, 'image/jpeg') # Provide filename and content type
            }

            print(f"Sending POST request to: {api_url}")
            print(f"Fruit type: {fruit_type}")
            print(f"Image file: {image_path}")

            response = requests.post(api_url, data=payload, files=files_to_upload, timeout=60) # Added timeout
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

            print(f"\nRequest successful! Status Code: {response.status_code}")
            print("Server JSON response:")
            prediction = response.json()
            print(prediction)
            return prediction

    except FileNotFoundError:
        # This case is already checked above, but kept for robustness.
        print(f"Error: Could not open image file at: {image_path}")
        return None
    except requests.exceptions.Timeout:
        print(f"\nError: The request to {api_url} timed out.")
        return None
    except requests.exceptions.ConnectionError:
        print(f"\nError: Could not connect to the server at {api_url}.")
        return None
    except requests.exceptions.HTTPError as http_err:
        print(f"\nHTTP error occurred: {http_err}")
        print(f"Status Code: {http_err.response.status_code}")
        print(f"Response Content: {http_err.response.text}")
        return None
    except requests.exceptions.RequestException as req_err:
        # Catch any other request-related errors
        print(f"\nError during request to {api_url}: {req_err}")
        return None
    except Exception as e:
        # Catch any other unexpected errors
        print(f"\nAn unexpected error occurred: {e}")
        return None

In [3]:
# --- Configuration ---
# API endpoint URL
API_ENDPOINT_URL: str = "https://xxx/predict/" # CHANGE `xxx`

# Path to the image file for prediction
# <<< CHANGE THIS PATH >>>
IMAGE_FILE_PATH: str = "image/anthracnose-of-mango-bangladesh-2.jpg"

# Crop type corresponding to the image (must match a key in index_mapping.json)
# <<< CHANGE THIS CROP TYPE >>>
FRUIT_TYPE: str = "mango" # Example: "Apple", "Tomato", "Potato", ...

In [4]:
# --- Main execution ---
if __name__ == "__main__":
    prediction_result = predict_disease(API_ENDPOINT_URL, IMAGE_FILE_PATH, FRUIT_TYPE)

    if prediction_result:
        # Process the prediction_result dictionary as needed
        print("\nPrediction process completed.")
    else:
        print("\nPrediction process failed.")

Sending POST request to: https://xxx/predict/
Fruit type: mango
Image file: image/anthracnose-of-mango-bangladesh-2.jpg

Error: Could not connect to the server at https://xxx/predict/.

Prediction process failed.
